In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
import plotly.express as px

df = pd.read_csv('dpt2020.csv', sep=';')

df['annais'] = pd.to_numeric(df['annais'], errors='coerce').astype('Int64')

df = df[(df['annais'] >= 1900) & (df['annais'] <= 2020)]

name_counts = df['preusuel'].value_counts()

color_palettes = [
    px.colors.qualitative.Plotly, 
    px.colors.qualitative.Set3, 
    px.colors.qualitative.D3,
    px.colors.qualitative.Pastel,
    px.colors.qualitative.Dark24
]

name_color_mapping = {}
color_index = 0

# color distribution
for idx, (name, count) in enumerate(name_counts.items()):
    if count > 10000:
        color_palette = color_palettes[0]
    elif count > 6000:
        color_palette = color_palettes[1]
    elif count > 3000:
        color_palette = color_palettes[2]
    elif count > 800:
        color_palette = color_palettes[3]
    else:
        color_palette = color_palettes[4]
    
    color_index %= len(color_palette)
    name_color_mapping[name] = color_palette[color_index]
    color_index += 1

df['color'] = df['preusuel'].map(name_color_mapping)

# grouped_df = df.groupby(['annais', 'preusuel', 'color'])['nombre'].sum().reset_index()
grouped_df = df.groupby(['annais', 'preusuel', 'color'])['nombre'].sum().reset_index()


# The 15 most popular name
def get_top_15(df):
    top_15 = df.sort_values(by='nombre', ascending=False).groupby('annais').head(15)
    return top_15

top_15_df = get_top_15(grouped_df)

# The 15 last popular name
def get_bottom_15(df):
    bottom_15 = df.sort_values(by='nombre', ascending=True).groupby('annais').head(15)
    return bottom_15

bottom_15_df = get_bottom_15(grouped_df)

def create_figure(year, data_df, title):
    frame_data = data_df[data_df['annais'] == year]
    data = []
    
    for _, row in frame_data.iterrows():
        data.append(go.Bar(
            x=[row['nombre']],
            y=[row['preusuel']],
            text=[row['nombre']],
            textposition='outside',
            orientation='h',
            marker=dict(color=row['color']),
            showlegend=False
        ))
    
    fig = go.Figure(
        data=data,
        layout=go.Layout(
            title=title,
            xaxis=dict(title='Nombre de naissances', titlefont=dict(size=12), tickfont=dict(size=10)),
            yaxis=dict(title='Prénom', titlefont=dict(size=12), tickfont=dict(size=10), autorange='reversed'),
            margin=dict(l=150, r=10, t=40, b=30)
        )
    )
    
    return fig

app = dash.Dash(__name__)

years = sorted(top_15_df['annais'].unique())

app.layout = html.Div(children=[
    html.H1(children='Visualisation des prénoms en France'),
    html.Div(children='''
        Comment les prénoms ont évolué en popularité et en impopularité de 1900 à 2020.
    '''),
    html.Button('Start', id='start-button', n_clicks=0),
    html.Button('Pause', id='pause-button', n_clicks=0),
    html.Button('Replay', id='replay-button', n_clicks=0),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # 1 second for each frame
        n_intervals=0,
        disabled=True 
    ),
    html.Div([
        dcc.Graph(id='top-bar-race-chart'),
        dcc.Graph(id='bottom-bar-race-chart')
    ], style={'display': 'flex', 'justify-content': 'space-around'})
])


@app.callback(
    [Output('interval-component', 'disabled'),
     Output('interval-component', 'n_intervals')],
    [Input('start-button', 'n_clicks'),
     Input('pause-button', 'n_clicks'),
     Input('replay-button', 'n_clicks')],
    State('interval-component', 'n_intervals')
)
def toggle_animation(start_clicks, pause_clicks, replay_clicks, n_intervals):
    ctx = dash.callback_context

    if not ctx.triggered:
        return True, n_intervals
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'start-button':
        return False, n_intervals
    elif button_id == 'pause-button':
        return True, n_intervals
    elif button_id == 'replay-button':
        return False, 0  


@app.callback(
    [Output('top-bar-race-chart', 'figure'),
     Output('bottom-bar-race-chart', 'figure')],
    Input('interval-component', 'n_intervals')
)
def update_figure(n):
    year = years[n % len(years)]
    top_fig = create_figure(year, top_15_df, f"Les prénoms les plus populaires en {year}")
    bottom_fig = create_figure(year, bottom_15_df, f"Les prénoms les moins populaires en {year}")
    return top_fig, bottom_fig


if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_26172\2909185771.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ASUS\AppData\Local\Temp\ipykernel_26172\2909185771.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
